In [15]:
import pandas as pd
import numpy as np

In [17]:
import joblib
model = joblib.load('model_chollo.pkl')
preprocessor = joblib.load('preprocessor.pkl')

In [25]:
def preprocesar_nuevos_datos(nuevos_datos):
    # Convertir a DataFrame si es un diccionario
    if isinstance(nuevos_datos, dict):
        nuevos_datos = pd.DataFrame([nuevos_datos])
    
    # Preprocesar los nuevos datos
    return preprocessor.transform(nuevos_datos)

In [19]:
def predecir_chollo(nuevos_datos):
    # Preprocesar los nuevos datos
    X_nuevo_prep = preprocesar_nuevos_datos(nuevos_datos)
    
    # Realizar predicción
    prediccion = model.predict(X_nuevo_prep)
    
    # Obtener probabilidades (para RandomForest y otros modelos que lo soporten)
    if hasattr(model, 'predict_proba'):
        probabilidad = model.predict_proba(X_nuevo_prep)[:, 1]  # Probabilidad de la clase 1 (chollo)
    else:
        probabilidad = None
    
    # Crear un resumen con los detalles
    if isinstance(nuevos_datos, dict):
        detalles = nuevos_datos.copy()
    else:
        detalles = nuevos_datos.iloc[0].to_dict() if len(nuevos_datos) == 1 else nuevos_datos.to_dict(orient='records')
    
    resultado = {
        'es_chollo': bool(prediccion[0] == 1),
        'probabilidad': float(probabilidad[0]) if probabilidad is not None else None,
        'detalles': detalles
    }
    
    return resultado

In [45]:
# Crear datos de prueba con las columnas adecuadas que espera el preprocessor
datos_prueba = {
    'Procesador': 'Apple M1',
    'RAM': 8.0,
    'Tipo RAM': 'DDR4',
    'Almacenamiento': 512.0,
    'Graficos': 'Intel Iris Xe Graphics',
    'Pantalla': 15.6,
    'Resolucion': '1920x1080',
    'Sistema Operativo': 'Windows 11 Home',
    'Bateria': 41.0,
    'Precio': 4650.0
}

# Realizar predicción
resultado = predecir_chollo(datos_prueba)

# Mostrar resultado
print(f"¿Es un chollo?: {resultado['es_chollo']}")
print(f"Probabilidad de ser chollo: {resultado['probabilidad']:.2f}")
print("\nDetalles del producto:")
for clave, valor in resultado['detalles'].items():
    print(f"- {clave}: {valor}")

¿Es un chollo?: True
Probabilidad de ser chollo: 0.67

Detalles del producto:
- Procesador: Apple M1
- RAM: 8.0
- Tipo RAM: DDR4
- Almacenamiento: 512.0
- Graficos: Intel Iris Xe Graphics
- Pantalla: 15.6
- Resolucion: 1920x1080
- Sistema Operativo: Windows 11 Home
- Bateria: 41.0
- Precio: 4650.0
